In [ ]:
from datetime import datetime, timedelta
from urllib.parse import urlencode
import polars as pl

def query_event_window(
    event_time,
    hours_before=2,
    hours_after=2,
    system='E',
    satellite_id=None,
    fields=None,
    latitude_min=None,
    latitude_max=None,
    longitude_min=None,
    longitude_max=None,
):
    """
    Query GNSS data from n hours before and m hours after an event time,
    optionally filtered by system, satellite, fields, and lat/lon bounds.
    
    Parameters:
    -----------
    event_time : str
        Event time in ISO format (e.g., "2025-07-29T23:24:52Z")
    hours_before : int
        Hours before event to query
    hours_after : int
        Hours after event to query
    system : str or list
        GNSS system(s), e.g., 'G', 'E', or ['G','E']
    satellite_id : int or list
        Specific satellite ID(s) to filter by
    fields : list
        List of fields to include in response
    latitude_min, latitude_max : float
        Latitude bounds (optional)
    longitude_min, longitude_max : float
        Longitude bounds (optional)
    
    Returns:
    --------
    polars.DataFrame or None
    """
    # Parse and compute window
    event_dt = datetime.fromisoformat(event_time.replace('Z', '+00:00'))
    start_dt = event_dt - timedelta(hours=hours_before)
    end_dt = event_dt + timedelta(hours=hours_after)
    start_str = start_dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    end_str = end_dt.strftime('%Y-%m-%dT%H:%M:%SZ')

    # Base URL
    base_url = 'https://arrow-api-dev.dev.earthscope.org/geometry_free_phase'

    # Query params
    params = {
        "timestamp_start": start_str,
        "timestamp_end": end_str,
    }

    if system is not None:
        params["system"] = [system] if isinstance(system, str) else system
    if satellite_id is not None:
        params["satellite_id"] = [satellite_id] if isinstance(satellite_id, int) else satellite_id
    if fields is not None:
        params["field"] = fields
    if latitude_min is not None:
        params["latitude_min"] = latitude_min
    if latitude_max is not None:
        params["latitude_max"] = latitude_max
    if longitude_min is not None:
        params["longitude_min"] = longitude_min
    if longitude_max is not None:
        params["longitude_max"] = longitude_max

    url = f"{base_url}?{urlencode(params, doseq=True)}"

    print(f"Querying data from {start_str} to {end_str}")

    # Download Arrow stream
    try:
        df = pl.read_ipc_stream(url)
        print(f"Retrieved {len(df)} records")
        return df
    except Exception as e:
        print(f"Error querying data: {e}")
        return None


In [ ]:
# Plot satellites offset by initial pierce point distance from event
import numpy as np
import pandas as pd
from math import radians, cos, sin, sqrt, atan2
import matplotlib.pyplot as plt

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points on Earth.
    Returns distance in kilometers.
    """
    R = 6371  # Earth's radius in kilometers
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    
    return distance

def plot_satellites_with_distance_offset(df_event, event_lat, event_lon, field='a_tec'):
    """
    Plot each satellite's data offset by its initial pierce point distance from event.
    Groups by time, stream_id, system, and satellite_id.
    
    Parameters:
    -----------
    df_event : polars.DataFrame
        DataFrame with satellite data
    event_lat : float
        Event latitude in degrees
    event_lon : float
        Event longitude in degrees
    field : str
        Field to plot (default: 'a_tec')
    """
    # Convert to pandas for easier manipulation
    df_pd = df_event.to_pandas()
    
    # Group by time, stream_id, system, and satellite_id and take the first record from each group
    # This handles any duplicate records
    df_grouped = df_pd.groupby(['timestamp', 'stream_id', 'system', 'satellite_id']).first().reset_index()
    
    # Calculate time offset from event time
    event_time = pd.to_datetime("2025-07-29T23:24:52Z")
    df_grouped['time_offset'] = (pd.to_datetime(df_grouped['timestamp']) - event_time).dt.total_seconds() / 3600  # hours
    
    # Create unique satellite identifier (combining system and satellite_id)
    df_grouped['satellite_key'] = df_grouped['system'] + '_' + df_grouped['satellite_id'].astype(str) + '_' + df_grouped['stream_id'].astype(str)
    
    # Get unique satellites
    satellites = df_grouped['satellite_key'].unique()
    
    # Calculate initial pierce point distances for each satellite
    satellite_distances = {}
    
    for sat in satellites:
        sat_data = df_grouped[df_grouped['satellite_key'] == sat]
        if len(sat_data) > 0:
            # Use the first record's pierce point
            first_record = sat_data.iloc[0]
            pp_lat = first_record['pp_latitude']
            pp_lon = first_record['pp_longitude']
            
            # Calculate distance from event to pierce point
            distance = haversine_distance(event_lat, event_lon, pp_lat, pp_lon)
            satellite_distances[sat] = distance
    
    # Create the plot
    plt.figure(figsize=(12, 8))
    
    # Color map for satellites
    colors = plt.cm.tab20(np.linspace(0, 1, len(satellites)))
    
    for i, sat in enumerate(satellites):
        sat_data = df_grouped[df_grouped['satellite_key'] == sat].sort_values('time_offset')
        
        if len(sat_data) > 0:
            # Get the distance offset for this satellite
            distance_offset = satellite_distances.get(sat, 0)
            
            # Offset the data by the distance
            offset_data = sat_data[field]*10000 + distance_offset
            
            # Plot with satellite-specific color
            plt.plot(sat_data['time_offset'], offset_data, 
                    label=f'{sat} ({distance_offset:.1f} km)', 
                    color=colors[i], alpha=0.8, linewidth=0, marker='.', markersize=0.5)
    
    plt.xlabel('Time Offset from Event (hours)')
    plt.ylabel(f'{field} + Distance Offset (km)')
    plt.title(f'Satellite {field} vs Time (Offset by Pierce Point Distance from Event)')
    # plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left') 
    plt.grid(True, alpha=0.3)
    plt.ylim(2000, 5000)
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"Event coordinates: ({event_lat:.3f}°, {event_lon:.3f}°)")
    print(f"Total unique satellite records: {len(df_grouped)}")
    print(f"Number of unique satellites: {len(satellites)}")
    print("\nSatellite distances from event:")
    for sat, dist in sorted(satellite_distances.items()):
        print(f"  {sat}: {dist:.1f} km")

# Example usage
event_lat = 52.512  # Replace with your event latitude
event_lon = -160.324  # Replace with your event longitude

# Make sure you have the required fields in your query
fields = ['timestamp', 'stream_id', 'system', 'satellite_id', 'a_tec', 'pp_latitude', 'pp_longitude']
df_event = query_event_window(
    event_time="2025-07-29T23:24:52Z",
    hours_before=-4,
    hours_after=8,
    system=["G", "E"],
    satellite_id=range(1, 32),
    fields=fields,
    latitude_min=16.0,
    latitude_max=24.0,
    longitude_min=-161.5,
    longitude_max=-154.0,
)

plot_satellites_with_distance_offset(df_event, event_lat, event_lon, field='a_tec')
